El objetivo de este notebook es implementar una red neuronal simplicial como se presentan en el articulo *Simplicial Neural Networks* de Stefania Ebli y Michaël Defferrard.

El Semantic Scholar Open Research Corpus (SSORC) es una base de datos que contiene informacion de mas de 39 millones de articulos academicos. Para nuestros objetivos, conviene pensar en la base de datos como un conjunto donde cada elemento es un diccionario asociado a un articulo con la siguiente estructura

```
paper = {
  'pid': el id del articulo,
  'authors': lista de los ids de los autores,
  '#cits': numero de veces que ha sido citado
}
```

En la base de datos real, cada diccionario contiene mas informacion (items) pero en la implementacion, esta solo se ocupa para hacer downsampling. Por ejemplo, se quitan todos los articulos que satisfacen una de las siguientes condiciones (entre otras):

* Falta el año de publicacion.
* Alguno de los autores tiene mas de un id asociado.
* No tienen referencias (i.e. no citan ningun otro articulo).
* Tienen mas de 10 autores.

Cabe recalcar que el downsampling requiere cuidado y es parte del objetivo de los scripts s2_2 y s2_4 del github.

El complejo de coautores $C$ es el complejo simplicial donde un
articulo con k autores esta representado por un $(k−1)$-simplejo.
Estamos interesados en las cocadenas dadas por $c(\{v_0,...,v_p\})$ = el número
total de citas de articulos que contienen a $v_0,...,v_p$ como autores.

El objetivo del preprocesamiento de datos es
1. Obtener subscomplejos de $C$ asociados a una seed = paper y a una cantidad fija de articulos n_papers.
2. Calcular la cocadena que nos interesa en cada subcomplejo como una lista de tensores de dimension $(1, M_d)$ con $M_d$ = el numero de simplejos de dimension $d$ en el subcomplejo.
3. Dañar la cocadena obtenida en el inciso anterior. Especificamente, reemplazar una cantidad fija de valores por la mediana de los valores que no quitamos.
4. Calcular la mascara de los valores que quitamos. Basicamente es una forma sistematica de saber que valores son reales y cuales son la media (para evaluar).
5. Calcular las matrices laplacianas y las matrices asociadas a los operadores frontera (para el subcomplejo fijo).

Por simplicidad, explicamos los detalles del preprocesamiento en el otro notebook.

In [ ]:
pip install gudhi

In [ ]:
import copy
import gudhi
import scipy
import torch
import random
import string

import numpy as np
import pandas as pd
import networkx as nx
import torch.nn as nn

# SSORC de juguete

* 5000 articulos
* 1000 autores
* 1-3 autores por articulo
* 1-20 citas por articulo


In [ ]:
def random_id(prefix, length):
    return prefix + ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

In [ ]:
paper_ids = list(set(random_id('P', 6) for _ in range(5000)))
author_ids = list(set(random_id('A', 6) for _ in range(1000)))

In [ ]:
ssorc = []
for pid in paper_ids:
  authors = random.sample(author_ids, random.randint(1, 3))
  num_cits = random.randint(1, 20)
  paper = {
      'pid': pid,
      'authors': authors,
      '#cits': num_cits
  }
  ssorc.append(paper)

In [ ]:
edges = [
    (paper['pid'], author)
    for paper in ssorc
    for author in paper['authors']
]
edges = pd.DataFrame(data=edges, columns=['paper','author'])
edges.head()

,paper,author
0,PUHJCG9,ALXBINP
1,PUHJCG9,AT0DCEQ
2,PUHJCG9,AB5VJYH
3,P2NS3MR,AFFPAXN
4,P2NS3MR,A8C6N4J


In [ ]:
papers = {paper['pid']: paper['#cits'] for paper in ssorc}
papers = pd.DataFrame.from_dict(data=papers, orient='index', columns=['#cits'])
papers.index.name = 'pid'
papers.head()

,#cits
pid,
PUHJCG9,1
P2NS3MR,12
P6ZDTPT,20
PSVXHS4,13
PZNBNX9,6


# Preprocesamiento

Descripcion de la funcion preprocessing: (los detalles se encuentran en el otro notebook)

Input:

* papers (data frame de pandas como se visualiza arriba)
* edges (data frame de pandas como se visualiza arriba)
* seed (un pid aleatorio)
* n_papers (el numero maximo de papers que permitimos en el subcomplejo - el cual se genera con BFS en la grafica edges)
* percent (el porcentaje de papers que quitamos al dañar la cocadena)

Output: Una lista de diccionarios (uno por seed) con los siguientes items

*	mask: una lista de listas (una por dimension) de indices que indican los valores no dañados.
*	target: lista de tensores (uno por dimension) que representan la cocadena *real* restringida al subcomplejo.
*	xs: lista de tensores (uno por dimension) que representan la cocadena dañada.
*	Ls: listas de matrices laplacianas normalizadas (una por dimension) asociadas al subcomplejo.

In [ ]:
def preprocessing(papers, edges, seed, n_papers, percent):

  new_papers = [seed]
  keep_papers = {seed}

  while len(keep_papers) < n_papers:
    new_authors = edges['author'][edges['paper'].isin(new_papers)]
    new_papers = edges['paper'][edges['author'].isin(new_authors)]

    new_papers_set = set(new_papers.values) - keep_papers
    if not new_papers_set:
      break

    keep_papers.update(new_papers_set)
    new_papers = list(new_papers_set)

  papers = papers.loc[list(keep_papers)]
  papers = papers.sort_index()

  edges = edges[edges['paper'].isin(papers.index)]
  edges = edges.sort_values('paper')
  edges = edges.reset_index(drop=True)

  authors = pd.DataFrame(edges['author'].unique(), columns=['author'])
  authors.sort_values('author', inplace=True)
  authors.set_index('author', inplace=True, verify_integrity=True)
  authors['aidx'] = np.arange(len(authors))

  papers['pidx'] = np.arange(len(papers))

  edges = edges.join(papers['pidx'], on='paper', how='right')
  edges = edges.join(authors['aidx'], on='author', how='right')
  edges.sort_index(inplace=True)

  biadjacency = scipy.sparse.coo_matrix(
      (np.ones(len(edges), dtype=bool),
      (edges['pidx'], edges['aidx']))
  )

  biadj_lil = biadjacency.tolil()

  st = gudhi.SimplexTree()
  for authors in biadj_lil.rows:
    st.insert(authors)

  D = st.dimension()

  pap_cits_by_authors = [dict() for _ in range(D+1)]
  for j, authors in enumerate(biadj_lil.rows):
    k = len(authors)
    pap_cits_by_authors[k-1].setdefault(frozenset(authors),[]).append(papers['#cits'].iloc[j])

  precochain = [dict() for _ in range(D+1)]
  for d in range(len(pap_cits_by_authors)-1, -1, -1):
    for simplex, values in pap_cits_by_authors[d].items():
      st_simplex = gudhi.SimplexTree()
      st_simplex.insert(simplex)
      for face, _ in st_simplex.get_skeleton(st_simplex.dimension()):
        face = frozenset(face)
        precochain[len(face)-1].setdefault(face, []).extend(pap_cits_by_authors[d][simplex])

  cochain = [dict() for _ in range(D+1)]
  for d in range(len(pap_cits_by_authors)-1, -1, -1):
    for simplex, values in pap_cits_by_authors[d].items():
      st_simplex = gudhi.SimplexTree()
      st_simplex.insert(simplex)
      for face, _ in st_simplex.get_skeleton(st_simplex.dimension()):
        face = frozenset(face)
        value = np.array(precochain[len(face)-1][face])
        cochain[len(face)-1][face] = int(np.sum(value))

  simplices = [dict() for _ in range(D+1)]
  for simplex, _ in st.get_skeleton(D):
    k = len(simplex)-1
    simplices[k][frozenset(simplex)] = len(simplices[k])

  simplices_list = [[None] * len(simplices[d]) for d in range(D+1)]
  for d in range(D+1):
    for simplex, idx in simplices[d].items():
      simplices_list[d][idx] = simplex

  subsimplices = [dict() for _ in range(D+1)]
  for d in range(D+1):
    d_simp_list = list(simplices[d].keys())
    m = int(np.ceil((len(d_simp_list)/100)*percent))
    random.shuffle(d_simp_list)
    subsimplices_list = d_simp_list[:m]
    for simplex in subsimplices_list:
      subsimplices[d][simplex] = simplices[d][simplex]

  mask = [list() for _ in range(D+1)]
  for d in range(D+1):
    known_simplices = list(set(simplices[d].keys()) - set(subsimplices[d].keys()))
    for simplex in known_simplices:
      mask[d].append(simplices[d][simplex])

  median_random = []
  for d in range(D+1):
    known_simplices = list(set(simplices[d].keys()) - set(subsimplices[d].keys()))
    median_random.append(np.median([cochain[d][simplex] for simplex in known_simplices]))

  damaged_cochain = copy.deepcopy(cochain)
  for d in range(D+1):
    for simplex in subsimplices[d].keys():
      damaged_cochain[d][simplex] = median_random[d]

  cochain_values = []
  for d in range(D+1):
    cochain_values_d = []
    for idx in range(len(simplices[d])):
      cochain_values_d.append(cochain[d][simplices_list[d][idx]])
    cochain_values.append(cochain_values_d)

  cochain_tensors = []
  for d in range(D+1):
    cochain_tensor = torch.zeros((1, len(cochain_values[d])), dtype=torch.float, requires_grad=False)
    cochain_tensor[0, :] = torch.tensor(cochain_values[d], dtype=torch.float, requires_grad=False)
    cochain_tensors.append(cochain_tensor)

  damaged_cochain_values = []
  for d in range(D+1):
    damaged_cochain_values_d = []
    for idx in range(len(simplices[d])):
      damaged_cochain_values_d.append(damaged_cochain[d][simplices_list[d][idx]])
    damaged_cochain_values.append(damaged_cochain_values_d)

  damaged_cochain_tensors = []
  for d in range(D+1):
    damaged_cochain_tensor = torch.zeros((1, len(damaged_cochain_values[d])), dtype=torch.float, requires_grad=False)
    damaged_cochain_tensor[0, :] = torch.tensor(damaged_cochain_values[d], dtype=torch.float, requires_grad=False)
    damaged_cochain_tensors.append(damaged_cochain_tensor)

  boundaries = []
  for d in range(1, D+1):
    values, idx_simplices, idx_faces = [], [], []
    for simplex, idx_simplex in simplices[d].items():
      for i, left_out in enumerate(np.sort(list(simplex))):
        values.append((-1)**i)
        idx_simplices.append(idx_simplex)
        idx_faces.append(simplices[d-1][simplex.difference({left_out})])
    assert len(values) == (d+1) * len(simplices[d])
    boundary = scipy.sparse.coo_matrix(
        (values, (idx_faces, idx_simplices)),
        dtype = np.float32,
        shape = (len(simplices[d-1]), len(simplices[d]))
    )
    boundaries.append(boundary)

  laplacians = list()
  up = scipy.sparse.coo_matrix(boundaries[0] @ boundaries[0].T)
  laplacians.append(up)
  for d in range(len(boundaries)-1):
    down = boundaries[d].T @ boundaries[d]
    up = boundaries[d+1] @ boundaries[d+1].T
    laplacians.append(scipy.sparse.coo_matrix(down + up))
  down = boundaries[-1].T @ boundaries[-1]
  laplacians.append(scipy.sparse.coo_matrix(down))

  for i in range(len(laplacians)):
    L = laplacians[i]
    bigeig = scipy.sparse.linalg.eigsh(L, k=1, which='LM', return_eigenvectors=False)[0]
    laplacians[i] = L * (1.0 / bigeig)

  def coo2tensor(A):
    idxs = torch.LongTensor(np.vstack((A.row, A.col)))
    vals = torch.FloatTensor(A.data)
    return torch.sparse_coo_tensor(idxs, vals, size=A.shape, requires_grad=False)

  Ls = [coo2tensor(laplacians[d]) for d in range(D+1)]

  sample = {
        'mask': mask,
        'target': cochain_tensors,
        'xs': damaged_cochain_tensors,
        'Ls': Ls,
  }

  return sample

# Nuestra base de datos (procesada)

* 500 articulos es la cota de articulos para los subcomplejos.
* 20% es el porcentaje de articulos que quitamos al dañar.

* 250 semillas para generar los subcomplejos asociados.
* 200 de estas son para entrenar.
* 50 para validar.


In [ ]:
n_papers = 500
percent = 20

n_samples = 250
seeds = random.choices(paper_ids, k=n_samples)
train_seeds = seeds[:200]
val_seeds = seeds[200:]

train_data = [preprocessing(papers, edges, seed, n_papers, percent) for seed in train_seeds]
val_data = [preprocessing(papers, edges, seed, n_papers, percent) for seed in val_seeds]

Sea $d$ una dimension fija y denotamos un simplejo arbitrario de dimension $d$ por $\sigma_m$. Recordemos que la convolucion esta dada por

$$
\mathcal{F}_p^{-1}(\varphi_\theta) \ast_p c = \sum_{i=0}^I \sum_{k=0}^K \theta_{i,k} T_k(\tilde{L}_d) c_i
$$
donde $I$ es el numero de input channels,  $T_k$ es el $k$-esimo polinomio de Chebyshev, $\tilde{L}_d$ es la matriz laplaciana de dimension $d$ normalizada, y $c_i = (c(\sigma_m)_i)_m$ es la matriz (vector columna) asociada a $c_i = \pi_i \circ c$.


La estructura de la siguiente funcion esta dada por:

Input:
* $x$ es un $(I,M)$ tensor (donde $M$ es el numero de simplejos de dimension $d$) con $x[i][m]$ = el valor de la cocadena en el $i$-esimo canal y $m$-esimo simplejo de dimension $d$.
* $L$ es la matriz laplaciana normalizada de dimension $d$.
* $K$ es el grado del polinomio de Chebyshev.

Output:
* $X$ es un $(I,M,K)$ tensor con $X[i][m][k] = T_k(\tilde{L}_d)c_i$ donde $c_i$ es el vector columna $(x[i][m])_m$ y .

In [ ]:
def chebyshev(x, L, K):

  (I, M) = x.shape

  X = []

  for i in range(I):
    Xi = []
    Xi.append(x[i, :].unsqueeze(1))
    if K > 1:
      Xi.append(L.mm(Xi[0]))
    for k in range(2,K):
      Xi.append(2*(L.mm(Xi[k-1])) - Xi[k-2])
    Xi = torch.cat(Xi,1)
    X.append(Xi.unsqueeze(0))

  X = torch.cat(X,0)

  assert(X.shape == (I, M, K))

  return X

# Capas convolucionales

Sea $d$ una dimension fija. Sea $I$ el numero de input channels, $O$ el numero de output channels, $K$ el grado del polinomio de Chebyshev, y $M$ el numero de simplejos de dimension $d$. Para un tensor $x_d \in \mathbb{R}^{I \times M}$

$$
C^d(x_d, \tilde{L}_d)[o, m] = \sum_{i=0}^{I-1} \sum_{k=0}^{K-1} \theta^{(d)}_{o,i,k} \left[ T_k(\tilde{L}_d) c_i \right]_m + b_o \in \mathbb{R}^{O \times M}
$$

Por simplicidad, en lo que sigue omitimos la $d$ pero es importante recordar que estas capas convolucionales operan sobre una dimension simplicial fija.

Es natural pensar en $C$ como una funcion de $\mathbb{R}^{I \times M}$ a $\mathbb{R}^{O \times M}$ (formalmente, esto no es preciso pues los parametros no estan fijos). En este caso, denotamos $C = C^{I,O}$. Esta notacion sera útil para describir la arquitectura modular de la red neuronal, donde cada capa actúa como una transformacion entre espacios de cocadenas de distintas dimensiones.

La funcion forward de la clase SimplicialConvolution regresa $C(x, \tilde{L})$ dado el input $(x, \tilde{L})$. De esta manera, la red (definida en la clase MySCNN a continuacion) aprende
1. Filtros convolucionales de dimension $(O, I, K)$.
2. Sesgos de dimension $(O, 1)$.

In [ ]:
class SimplicialConvolution(nn.Module):

  def __init__(self, I, O, K):

    super().__init__()

    self.I = I
    self.O = O
    self.K = K

    self.theta = nn.parameter.Parameter(torch.randn(self.O, self.I, self.K))
    self.bias = nn.Parameter(torch.zeros(self.O))

  def forward(self, x, L):

    X = chebyshev(x, L, self.K)
    y = torch.einsum('imk,oik->om', (X, self.theta))

    return y + self.bias.view(-1, 1)

# Estructura de la red

Para cada dimension $d = 0, 1, 2$, se aplican tres capas secuenciales de convolucion simplicial:

$$
C_1 \rightarrow \text{ReLU} \rightarrow C_2 \rightarrow \text{ReLU} \rightarrow C_3
$$

Mas precisamente, (y especificando las dimensiones) la red neuronal opera asi:

$$
\mathbb{R}^{I \times M} \xrightarrow{C^{I,O}} \mathbb{R}^{O \times M} \xrightarrow{\text{ReLU}} \mathbb{R}^{O \times M} \xrightarrow{C^{O,O}} \mathbb{R}^{O \times M} \xrightarrow{\text{ReLU}} \mathbb{R}^{O \times M} \xrightarrow{C^{O,I}} \mathbb{R}^{I \times M}
$$

(sin activacion final).

In [ ]:
class MySCNN(nn.Module):

  def __init__(self, colors=1):

    super().__init__()

    self.colors = colors
    num_filters = 10
    K = 5

    self.C01 = SimplicialConvolution(colors, num_filters, K)
    self.C02 = SimplicialConvolution(num_filters, num_filters, K)
    self.C03 = SimplicialConvolution(num_filters, colors, K)

    self.C11 = SimplicialConvolution(colors, num_filters, K)
    self.C12 = SimplicialConvolution(num_filters, num_filters, K)
    self.C13 = SimplicialConvolution(num_filters, colors, K)

    self.C21 = SimplicialConvolution(colors, num_filters, K)
    self.C22 = SimplicialConvolution(num_filters, num_filters, K)
    self.C23 = SimplicialConvolution(num_filters, colors, K)

  def forward(self, xs, Ls):

    out0 = self.C01(xs[0], Ls[0])
    out1 = self.C11(xs[1], Ls[1])
    out2 = self.C21(xs[2], Ls[2])

    out0 = self.C02(nn.LeakyReLU()(out0), Ls[0])
    out1 = self.C12(nn.LeakyReLU()(out1), Ls[1])
    out2 = self.C22(nn.LeakyReLU()(out2), Ls[2])

    out0 = self.C03(nn.LeakyReLU()(out0), Ls[0])
    out1 = self.C13(nn.LeakyReLU()(out1), Ls[1])
    out2 = self.C23(nn.LeakyReLU()(out2), Ls[2])

    return [out0, out1, out2]

# Entrenamiento

Ocupamos el mismo optimizador y criterio de perdida que en el github. Especificamente,

```
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.L1Loss(reduction='sum')
```

Para cada sample en data, entrenamos el modelo n_epochs=20 epocas.


In [ ]:
def train(data, n_epochs):

  model = MySCNN(colors=1)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.L1Loss(reduction='sum')

  for sample_idx, sample in enumerate(data):
    mask = sample['mask']
    target = sample['target']
    xs = sample['xs']
    Ls = sample['Ls']

    print(f"\nTraining on Sample {sample_idx}")

    for epoch in range(n_epochs):

      model.train()
      optimizer.zero_grad()
      ys = model(xs, Ls)

      loss = torch.tensor(0.0)
      for d in range(3):
        loss += criterion(ys[d][0, mask[d]], target[d][0, mask[d]])

      loss.backward()
      optimizer.step()

      if epoch in [0, n_epochs - 1]:
        print(f"  Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

  return model

In [ ]:
trained_model = train(train_data, n_epochs=20)


Training on Sample 0
  Epoch 1/20, Loss: 1091734.8750
  Epoch 20/20, Loss: 710788.6875

Training on Sample 1
  Epoch 1/20, Loss: 488010.4688
  Epoch 20/20, Loss: 339703.1875

Training on Sample 2
  Epoch 1/20, Loss: 847596.7500
  Epoch 20/20, Loss: 430110.4062

Training on Sample 3
  Epoch 1/20, Loss: 183754.0625
  Epoch 20/20, Loss: 164911.2969

Training on Sample 4
  Epoch 1/20, Loss: 326338.0000
  Epoch 20/20, Loss: 257349.1562

Training on Sample 5
  Epoch 1/20, Loss: 233034.2344
  Epoch 20/20, Loss: 191583.2344

Training on Sample 6
  Epoch 1/20, Loss: 278451.5625
  Epoch 20/20, Loss: 225196.6562

Training on Sample 7
  Epoch 1/20, Loss: 213731.7812
  Epoch 20/20, Loss: 179906.7500

Training on Sample 8
  Epoch 1/20, Loss: 87093.8203
  Epoch 20/20, Loss: 62725.1445

Training on Sample 9
  Epoch 1/20, Loss: 211550.2188
  Epoch 20/20, Loss: 148363.6250

Training on Sample 10
  Epoch 1/20, Loss: 137412.0938
  Epoch 20/20, Loss: 121661.0469

Training on Sample 11
  Epoch 1/20, Loss: 

# Evaluacion

Los conceptos de accuracy y absolute errors definidos en la funcion evaluate coinciden con los definidos en el articulo. Especificamente,

*A missing value is considered to be correctly imputed if the imputed value differs by at most 10% from the true value. The accuracy is the percentage of missing values that has been correctly imputed and the absolute error is the magnitude of the difference between the imputed and true value.*


In [ ]:
def evaluate(model, data, criterion):

  total_loss = 0.0
  total_abs_error = 0.0
  total_correct = 0
  total_count = 0

  model.eval()

  with torch.no_grad():

    for sample in data:

      mask = sample['mask']
      target = sample['target']
      xs = sample['xs']
      Ls = sample['Ls']

      ys = model(xs, Ls)

      for d in range(3):

        preds = ys[d][0]
        trues = target[d][0]
        m = mask[d]

        loss = criterion(preds[m], trues[m])
        total_loss += loss.item()

        abs_diff = torch.abs(preds[m] - trues[m])
        total_abs_error += abs_diff.sum().item()

        correct = (abs_diff <= 0.1 * torch.abs(trues[m])).sum().item()
        total_correct += correct
        total_count += len(m)

  accuracy = total_correct / total_count if total_count else 0
  mean_abs_error = total_abs_error / total_count if total_count else 0

  return total_loss, accuracy, mean_abs_error

In [ ]:
val_loss, val_acc, val_mae = evaluate(trained_model, val_data, nn.L1Loss(reduction='sum'))

print(f"\nFinal Evaluation on Validation Set:")
print(f"Validation Loss: {val_loss:.2f}")
print(f"Validation Accuracy: {val_acc*100:.2f}%")
print(f"Mean Absolute Error: {val_mae:.4f}")


Final Evaluation on Validation Set:
Validation Loss: 259534.72
Validation Accuracy: 82.47%
Mean Absolute Error: 1.1433
